In [2]:
import cv2
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm

### Define paths

In [3]:
in_path = Path('./data/in/videos')
out_path = Path('./data/out/videos')

### Utilities

In [4]:
def show_image(img):
    plt.imshow(img, cmap='gray')
    plt.show()

def save_image(path, img):
    cv2.imwrite(path, img)

In [5]:
def manage_and_create_paths(image_path, out_path):
    res = out_path / image_path.stem
    if not res.parent.exists():
        res.mkdir(parents=True, exist_ok=True)
    return res

def create_output_path(path, out_path):
    return str(manage_and_create_paths(path, out_path)) + '/' + path.name

### Extract frames

In [19]:
for p in (in_path.rglob('*.mp4')):
    print(p)
    path = manage_and_create_paths(p, out_path)
    cap = cv2.VideoCapture(str(p))
    fps = cap.get(cv2.CAP_PROP_FPS)
    print('frames per second: ',fps)
    l = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print('total number of frames: ', l)
    
    for f in tqdm(range(l)):
        cap.set(cv2.CAP_PROP_POS_FRAMES, f)
        ret, frame = cap.read()
        res_path = './' + str(path) + f'/frame_{f}.jpg'
        save_image(res_path, frame)

frames per second:  29.97002997002997
total number of frames:  2196


100%|██████████| 2196/2196 [00:44<00:00, 49.80it/s]


## In case multiprocessing works:

In [ ]:
import cv2
from pathlib import Path
from tqdm import tqdm
from multiprocessing import Pool, cpu_count

def process_video(video_path, out_path):
    cap = cv2.VideoCapture(str(video_path))
    fps = cap.get(cv2.CAP_PROP_FPS)
    l = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frames = []
    for f in range(l):
        cap.set(cv2.CAP_PROP_POS_FRAMES, f)
        ret, frame = cap.read()
        frames.append((f, frame))
    cap.release()

    path = manage_and_create_paths(video_path, out_path)
    for frame_info in frames:
        f, frame = frame_info
        res_path = str(path) + f'/frame_{f}.jpg'
        save_image(res_path, frame)

def process_videos(video_paths, out_path):
    with Pool(cpu_count()) as pool:
        pool.starmap(process_video, [(video_path, out_path) for video_path in video_paths])

# Assuming manage_and_create_paths and save_image are defined elsewhere

# Main code
video_paths = list(in_path.rglob('*.mp4'))
process_videos(video_paths, out_path)
